## Clean up CSV files
read csv files from folder and add columns to each file with new entries based on the previous columns.

Transcript	IU_number	start_time	end_time	speaker	IU	
AAE_MF_002_Stella	1	02:04.1	02:09.1	INT: Sylvie	So do you do you did you went to the school chiao zhou like all the way 	



In [ ]:
import os
import re
import csv
import pandas as pd
import unicodedata, itertools, sys

#folder_path = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\data\\test"

"""
#Accented Australian English data
input_folder = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\data\\AllFiles_Accented_English"
output_folder = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\data\\AllFiles_Accented_English_clean"
statsFile = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\data\\wordCount_AAE.csv"

"""
# Sydney Speaks data
input_folder = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\data\\AllFiles_SydneySpeaks"
output_folder = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\data\\AllFiles_SydneySpeaks_clean"
statsFile = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\data\\wordCount_SydneySpeaks.csv"


"""
#Accented Australian English data
input_folder = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\data\\test"
output_folder = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\data\\test_clean"
statsFile = "C:\\Users\\barth\\Documents\\LDACA\\data_cleaning\\data\\wordCount_test.csv"
"""

def clean_speaker(speaker):
    speaker = speaker.replace("LaBB-CAT_", "")
    speaker = speaker.strip()
    return speaker


def clean_text(text):
    text = text.strip()
    text = text.replace("‚Äô", "'")
    text = text.replace("…", "...")
    text = text.replace("’", "'")
    return text


def handle_file(file):
    input_file_path = os.path.join(input_folder, file)
    output_file_path = os.path.join(output_folder, file)

    rowCount = 0
    #print (input_file_path)
    with open(input_file_path, mode='r', encoding="utf-8") as input_csv_file:

        # for rows that will be deleted because they only contain "." etc. at the end of the file
        deleteList = []

        csv_reader = csv.reader(input_csv_file)

        # skip the header
        next(csv_reader)

        # Process the data as needed
        processed_data = [["Transcript","IU_number","start_time","end_time","speaker", "IU"]]
        
        for row in csv_reader:

            # clean the speaker
            row[5] = clean_speaker(row[5])

            # clean the text
            row[6] = clean_text(row[6])

            # remove all rows which only have a dot . in the UI column
            if row[6] != ".":

                rowCount += 1
                row[2] = str(rowCount)
                processed_data.append(row[1:7])


            # find the tailing cassette and period info
            if ("CASSETTE CHANGE" in row[6]) or (row[6] == "."):
                if rowCount not in deleteList:
                    deleteList.append(rowCount) 
            else:
                # reset the list if there is a row which actually has content
                deleteList = []


    # remove all tailing Cassette change and period info            
    for each in processed_data[1:]:
        if int(each[1]) in deleteList:
            processed_data.remove(each)

            
    with open(output_file_path, mode='w', newline='', encoding='utf-8') as output_csv_file:

        csv_writer = csv.writer(output_csv_file)

        for row in processed_data:
            #print(row)
            csv_writer.writerow(row)
            
            pattern = "\w*- "
            pattern2 = " -\w*"
            if re.match(pattern, row[5]):
                print (row)
            #elif re.match(pattern2, row[5]):
            #    print (row)

   
# Function to read and save CSV files
def read_and_save_csv(input_folder, output_folder):

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.csv'):
                handle_file(file)
            
# Call the function to read and save CSV files
read_and_save_csv(input_folder, output_folder)

print (" +++ DONE +++")

## Count words after cleaning string

In [ ]:
# counts the words after cleaning the string


file_list = os.listdir(output_folder)

typeDict = {}

randomSample = 1
sample = []

# Filter out only CSV files
csv_files = [file for file in file_list if file.endswith('.csv')]

dataAll = {}
# Process CSV files

def remove_spaces_before_punctuation(input_string):
    return re.sub(r'\s+([,.!?;:])', r'\1', input_string)

def remove_single_letter_stutter(input_string):
    return re.sub(r'[A-Za-z]*[-~] ', '', input_string)

def filter_words(words_list):
    filtered_list = [word for word in words_list if all(c.isalnum() or c in '~=!.,:;[]' for c in word)]
    #print (filtered_list)
    return filtered_list

def remove_non_alphanumeric_strings(input_list):
    cleaned_list = [item for item in input_list if any(c.isalnum() for c in item)]
    return cleaned_list

def split_string_with_punctuation(input_string):
    
    pattern = r'\s+|\''
    """Use regex to split the string by words and punctuation"""
    words = re.split(pattern, input_string)
    #print (words)
    words = filter_words(words)
    words = remove_non_alphanumeric_strings(words)
    return words

def find_strings_with_symbols(string_list, symbol_list):
    # find all strings from list which don't contain any of the symbols in the other list
    filtered_strings = [s for s in string_list if not any(symbol in s for symbol in symbol_list)]
    return filtered_strings

def remove_parentheses(wordString):
    # remove all parenteses 
    if ("(" in wordString) and (")" in wordString):
        #print ('---------------------------')
        #print (wordString)
        res = re.findall(r'\(.*?\)', wordString)
        #print (wordString)
        for x in res:
            wordString = wordString.replace(x, "")
        #print (wordString)
    return wordString

def remove_tags(wordString):
    # remove all <This  and that>
    if "<" in wordString:
        #print ('---------------------------')
        #print (wordString)

        res = re.findall(r"<\S*", wordString)

        for x in res:
            wordString = wordString.replace(x, "")
        #print (wordString)

    if ">" in wordString:
        #print ('---------------------------')
        #print (wordString)
        res = re.findall(r"\S*>", wordString)

        for x in res:
            wordString = wordString.replace(x, "")
        #print (wordString)
    return wordString


def replace_items(inString):
    replaceDict = {"\t": "", 
                   "\n": "",
                   "’": "'",
                   "...": "",
                   "…": "",
                   "!": "",}

    for k, v in replaceDict.items():
        inString = inString.replace(k, v)
        
    return inString


def clean_words(rawList):
            
    newList = []

    for each in rawList:
        pattern = r"!\w+[^A-Za-z0-9_]"

        matches = re.findall(pattern, each)
        if len(matches) > 0:
            newList.append(matches[0][1:])
        else:
            newList.append(each)


    return newList

     
def filter_words(rawList):
    # filter out all words from list which have one of the sybmols in it
    # symbols to be filtered out:
    symbols = [
               "...",
               "..",
               "(N.N)",
               "!",
               "X ",
               "(",
               ")",
               "<",
               ">",
               "%"]


    rawList = find_strings_with_symbols(rawList, symbols)
    
    return rawList
   
def string_replace(outString):
    # replacement of some characters
    replaceDict = {"[": "", 
                   "]": "",
                   "=": "",
                   "~": "",
                   "@": "",
                   "--": "", 
                   "- ": " ",
                   " -": " ",
                   "NA": ""}


    for k, v in replaceDict.items():
        outString = outString.replace(k, v)
    return outString

def printChange(old, new):
    if old != new:
        print (str(old))
        print (str(new))
        print ("---------------------------------")
        
def remove_square_brackets(outString):
    pattern = r'\[[A-Za-z]-\]'
    outString = re.sub(pattern, '', outString) 
    return outString
        

def remove_numbers(wordString):
    return re.sub(r'\d', '', wordString)
    
def final_punctuation(wordString):
    wordString = wordString.replace(" ?", "?")
    return re.sub(r'\s+([.,!?])', r'\1', wordString)
    
letterList = []
for csv_file in csv_files:
    
    dataAll[csv_file] = {}
    
    csv_file_path = os.path.join(output_folder, csv_file)

    with open(csv_file_path, 'r', newline='', encoding="utf8") as input_file:
        
        csv_reader = csv.reader(input_file)
        
        # Read existing headers
        existing_headers = next(csv_reader)
        
        for row in csv_reader:
            randomSample += 1
            
            if row[4].startswith("PNT"):
                
                if row[4] not in dataAll[csv_file]:
                        dataAll[csv_file][row[4]] = 0

                wordString = row[5].strip()
                w1 = wordString
                
                
                wordString = remove_single_letter_stutter(wordString) # and then y- you can get...
                w2 = wordString
                
                
                
                wordString = remove_parentheses(wordString) # if my (Mike) older broth...
                w3 = wordString
                

                wordString = remove_tags(wordString) # and <then> we went...
                w4 = wordString
                

                wordString = replace_items(wordString) # replace individual symbols
                w5 = wordString
                
                
                wordString = remove_numbers(wordString)
                w6 = wordString
                
                

                wordString = wordString.strip()
                w7 = wordString




                # create List of words
                rawList = wordString.split()
                w8 = rawList

                
                rawList = clean_words(rawList)
                w9 = rawList


                rawList = filter_words(rawList)
                w10 = rawList



                # combine to string again
                outString = " ".join(rawList)
                w11 = outString
                
                
                outString = string_replace(outString)
                w12 = outString

                
                outString = final_punctuation(outString)
                w13 = outString

                
                outString = outString.strip()
                w14 = outString

                
                outString = remove_square_brackets(outString)
                w15 = outString
                
                
                
                #printChange(w1, w15)

                """for letter in outString:
                    if not letter.isalnum():
                        if letter not in letterList:
                            letterList.append(letter)"""
                # ['.', ',', "'", '?', '-', '&', '‑', ':', '≤', '\\', '}']
                for each in ['.', ',', '?', '&', '‑', ':', '≤', '\\', '}']:
                    outString = outString.replace(each, "")
                # now count the tokens
                
                
                w16 = outString
                
                #if "-" in w16:
                #    printChange(w1, w16)
                
                #printChange(w1, w16)
                #printChange(w1, w16)
                
                
                #if randomSample % 5  == 0:
                #    printChange(w1, w16)
                
                printChange(w1, w4)
                
                
                sample.append([w1, w16])
                
                outList = outString.split()

                
                for token in outList:
                     
                    if (len(token) == 1) and token not in "Ia":
                        pass
                    
                    else:
                        dataAll[csv_file][row[4]] += 1
                        
                        if token not in typeDict:
                            typeDict[token] = 1
                        else:
                            typeDict[token] += 1

                
typeDict_sorted = dict(sorted(typeDict.items()))

with open(statsFile, 'w', encoding="utf-8") as outf:
    for k, v in dataAll.items():
        for a, b in v.items():
            outf.write(str(k) + "," + str(a) + "," + str(b)+ "\n")
       
with open(statsFile, 'a', encoding="utf-8") as outf:   
    outf.write("\n\n\n")
    for c,d in typeDict_sorted.items():
        outf.write(str(c) + "    " + str(d) + "\n")
    
    for each in sample:
        outf.write(each[0] + "\n" + each[1] + "\n----------------------------\n")

print (letterList) 
print("+++ DONE +++")

In [67]:
if  '-' == '‑':
    print ("Yes")